In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


In [2]:
cols_plays = ["game_id", "eventmsgtype", "period", "pctimestring", "scoremargin", "player1_id", "player1_name"]
dtypes_plays = {
    "game_id": "Int32",
    "eventmsgtype": "UInt8",
    "period": "UInt8",
    "player1_id": "Int32",
    "player1_name": "object",
    "pctimestring": "object",
    "scoremargin": "object"
}


In [3]:
def cargar_jugadas(path="play_by_play.csv"):
    return pd.read_csv(path, usecols=cols_plays, dtype=dtypes_plays)


In [4]:
df = cargar_jugadas("play_by_play.csv")


In [ ]:
def filtrar_jugadas_clutch(df):
    df = df.copy()
    df["scoremargin"] = pd.to_numeric(df["scoremargin"], errors="coerce").astype("Int16")

    # Convertir a segundos
    mask_time = df["pctimestring"].str.match(r"^\d{1,2}:\d{2}$", na=False)
    mins = pd.to_numeric(df.loc[mask_time, "pctimestring"].str.split(":").str[0], errors="coerce")
    secs = pd.to_numeric(df.loc[mask_time, "pctimestring"].str.split(":").str[1], errors="coerce")
    df.loc[mask_time, "seconds_remaining"] = mins * 60 + secs

    clutch = df[
        (df["period"] >= 4) &
        (df["seconds_remaining"] <= 300) &
        (df["scoremargin"].abs() <= 5)
    ].copy()

    return clutch
